## PDI - Phuture DeFi Index (Multi-chain) - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["decentralized-finance-defi"]

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'polygon-pos',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['the-graph','staked-ether','injective-protocol','chainlink','seedify-fund']
    )
liquidity

Excluding chainflip, pricing data available only for 118 < 180 days
Excluding syncus, pricing data available only for 98 < 180 days
Excluding stakewise-v3-oseth, pricing data available only for 100 < 180 days
Excluding vertex-protocol, pricing data available only for 120 < 180 days
0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
0xa0b862f60edef4452f25b4160f177db44deb6cf1 arbitrum-one
0x7205705771547cf79201111b4bd8aaf29467b9ec polygon-pos
0xd7c49cee7e9188cca6ad8ff264c1da2e69d4cf3b ethereum
0x9046d36440290ffde54fe0dd84db8b1cfee9107b optimistic-ethereum
0x9eaf8c1e34f05a589eda6bafdf391cf6ad3cb239 base
0xab36452dbac151be02b16ca17d8919826072f64a base
0x77f56cf9365955486b12c4816992388ee8606f0e polygon-pos
0xaf8ca653fa2772d58f4368b0a71980e9e3ceb888 optimistic-ethereum
0xd58d345fd9c82262e087d2d0607624b410d88242 arbitrum-one
0xe3322702bedaaed36cddab233360b939775ae5f1 polygon-pos
0xd3cc9d8f3689b83c91b7b59cab4946b063eb894a ethereum
0xfd4330b0312fdeec6d4225075b82e00493ff2e3f base
0x1e1085efaa6

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,fantom,optimistic-ethereum,polygon-pos,optimal chain,best slippage,best slippage chain
id,,,,,,,,,,,
tbtc,-0.003266,NaN,-0.049718,NaN,-0.002495,NaN,-0.347900,-0.045013,arbitrum-one,-0.002495,ethereum
uniswap,-0.005771,NaN,NaN,-0.053606,-0.007791,NaN,-0.886532,-0.096801,arbitrum-one,-0.005771,arbitrum-one
maker,NaN,-0.999897,NaN,NaN,-0.006433,NaN,NaN,-0.923204,ethereum,-0.006433,ethereum
pancakeswap-token,NaN,NaN,NaN,-0.007631,NaN,NaN,NaN,NaN,binance-smart-chain,-0.007631,binance-smart-chain
gains-network,-0.008485,NaN,NaN,NaN,NaN,NaN,NaN,-0.199787,arbitrum-one,-0.008485,arbitrum-one
...,...,...,...,...,...,...,...,...,...,...,...
truefi,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,None,-1.000000,ethereum
vvs-finance,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,None,-1.000000,ethereum
orion-protocol,NaN,NaN,NaN,-1.000000,-1.000000,NaN,NaN,NaN,None,-1.000000,binance-smart-chain


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
uniswap,UNI,8123838409,0.302535,19827,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
maker,MKR,2691861472,0.138247,9060,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ethereum
lido-dao,LDO,2099489754,0.107824,7067,0xc3c7d422809852031b44ab29eec9f1eff2a58756,polygon-pos
aave,AAVE,1634997849,0.083969,5503,0xd6df932a45c0f255f85145f286ea0b292b21c90b,polygon-pos
ribbon-finance,RBN,1270864231,0.065268,4278,0x6123b0049f904d730db3c36a31167d9d4121fa6b,ethereum
havven,SNX,1190727307,0.061153,4008,0x8700daec35af8ff88c16bdf0418774cb3d7599b4,optimistic-ethereum
pancakeswap-token,CAKE,890634651,0.045741,2998,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,binance-smart-chain
woo-network,WOO,762239281,0.039147,2566,0x4691937a7508860f876c9c0a2a617e7d9e945d4b,ethereum
1inch,1INCH,576998988,0.029633,1943,0x111111111117dc0aa78b770fa6a738034120c302,ethereum


In [8]:
from backtester import BackTester, convert_results_table

bt = BackTester(convert_results_table(results),['defipulse-index'])

data, chart = bt.assess_different_rebalancing_periods([30,60,90],0.02)

chart

/Users/olivermehr/Documents/GitHub/rebalancing-data/PDI/../backtester.py:55: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.



defipulse-index


/Users/olivermehr/.local/share/virtualenvs/rebalancing-data-VGaZtODH/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [9]:
data

,Index cum-return w/ 30 days rebalancing window,Index cum-return w/ 60 days rebalancing window,Index cum-return w/ 90 days rebalancing window,defipulse-index-cum-rtn
2022-03-18,0.000000,0.000000,0.000000,NaN
2022-03-19,0.028376,0.028376,0.028376,0.000000
2022-03-20,0.050514,0.050514,0.050514,0.030899
2022-03-21,0.005068,0.005068,0.005068,-0.018022
2022-03-22,0.040894,0.040894,0.040894,-0.013608
...,...,...,...,...
2024-03-15,0.098830,0.329346,0.433974,-0.133959
2024-03-16,0.035496,0.252726,0.351323,-0.178286
2024-03-17,-0.026581,0.177627,0.270313,-0.232816
2024-03-18,0.029292,0.245221,0.343227,-0.189334
